# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [10]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import islice

In [11]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [12]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [13]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [14]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

### a) How many faculty taught COSI courses last year?

In [15]:
cosi_courses = len({c['instructor'] for c in courses if c['subject'] == 'COSI' })
cosi_courses

27

### b) What is the total number of students taking COSI courses last year?

In [16]:
enrolled_cosi = 0
for c in courses:
    if c['subject'] == 'COSI':
        enrolled_cosi += c['enrolled']
enrolled_cosi

2223

### c) What was the median size of a COSI course last year (counting only those courses with at least 10 students)?

In [17]:
median_cosi = []
for c in courses:
    if c['subject'] == 'COSI' and c['enrolled'] >= 10:
        median_cosi.append(c['enrolled'])
np.median(median_cosi)

37.0

### d) create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [18]:
subject_enrolled = {}
for c in courses:
    if c['subject'] in subject_enrolled:
        subject_enrolled[c['subject']] += c['enrolled']
    else:
        subject_enrolled[c['subject']] = c['enrolled']

In [19]:
subject_students = []
for subject, students in subject_enrolled.items():
    subject_students.append((subject, students))

In [20]:
subject_students.sort(key=lambda x: x[1], reverse=True)
subject_students[0:10]

[('HS', 5318),
 ('BIOL', 3085),
 ('BUS', 2766),
 ('HWL', 2734),
 ('CHEM', 2322),
 ('ECON', 2315),
 ('COSI', 2223),
 ('MATH', 1785),
 ('PSYC', 1704),
 ('ANTH', 1144)]

### e) Do the same as in (d) but print the top 10 subjects in terms of number of courses offered.

In [21]:
course_subjects = defaultdict(set)
for c in courses:
    course_subjects[c['subject']].add(c['name'])

In [22]:
subj_courses = []
for subject, course in course_subjects.items():
    subj_courses.append((subject, (len(course))))

In [24]:
subj_courses.sort(key=lambda x: x[1], reverse=True)
subj_courses[0:10]

[('HS', 170),
 ('ENG', 67),
 ('BIOL', 62),
 ('BUS', 58),
 ('ANTH', 56),
 ('MUS', 56),
 ('PSYC', 55),
 ('MATH', 53),
 ('ECON', 49),
 ('FA', 49)]

### f) do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject

In [25]:
faculty_dict = defaultdict(set)
for c in courses:
    faculty_dict[c['subject']].add(c['instructor'][2])

In [26]:
subj_faculty = []
for subject, profs in faculty_dict.items():
    subj_faculty.append((subject, (len(profs))))

In [28]:
subj_faculty.sort(key=lambda x: x[1], reverse=True)
subj_faculty[0:10]

[('HS', 84),
 ('BIOL', 67),
 ('ECON', 52),
 ('BCHM', 49),
 ('HIST', 47),
 ('BUS', 46),
 ('BCBP', 46),
 ('HWL', 42),
 ('MATH', 37),
 ('NEJS', 37)]

### g) list the top 20 faculty in terms of number of students they taught

In [29]:
faculty_students = defaultdict(list)
for c in courses:
    faculty_students[c['instructor'][0:2]].append(c['enrolled'])

In [30]:
sorted_faculty_students = ({k: v for k, v in sorted(faculty_students.items(),
                                                   key=lambda item: len(item[1]), reverse=True)})

In [31]:
x = 0
for key, value in islice(sorted_faculty_students.items(), 0, 20):
    x+=1
    print(f"#{x}", key, ": ", "num students: ", len(value))

#1 ('Suzanne G.', 'Paradis') :  num students:  51
#2 ('Leslie Claire', 'Griffith') :  num students:  49
#3 ('Kene Nathan', 'Piasta') :  num students:  49
#4 ('Sacha', 'Nelson') :  num students:  47
#5 ('Seth', 'Fraden') :  num students:  47
#6 ('Paul', 'DiZio') :  num students:  47
#7 ('John', 'Lisman') :  num students:  46
#8 ('Michael', 'Rosbash') :  num students:  45
#9 ('Donald B.', 'Katz') :  num students:  45
#10 ('Robert W', 'Sekuler') :  num students:  45
#11 ('Arthur', 'Wingfield') :  num students:  44
#12 ('Karen', 'Hansen') :  num students:  41
#13 ('Lizbeth', 'Hedstrom') :  num students:  40
#14 ('Avital', 'Rodal') :  num students:  39
#15 ('Susan J', 'Birren') :  num students:  39
#16 ('James R', 'Lackner') :  num students:  39
#17 ('Angela', 'Gutchess') :  num students:  39
#18 ('Ellen', 'Schattschneider') :  num students:  38
#19 ('Faith Lois', 'Smith') :  num students:  38
#20 ('Paul', 'Miller') :  num students:  36


### h) list the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number)

In [32]:
courses_enrolled = {}
for c in courses:
    courses_enrolled[(c['subject'], c['coursenum'])] = c['enrolled']

In [33]:
sorted_courses_enrolled = {k: v for k, v in sorted(courses_enrolled.items(), key=lambda item: item[1], reverse=True)}

In [34]:
i = 0
for key, value in islice(sorted_courses_enrolled.items(), 0, 20):
    i+=1
    print(f"#{i}", key, ": ", "enrolled: ", value)

#1 ('HWL', '1-PRE') :  enrolled:  784
#2 ('BIOL', '42A') :  enrolled:  186
#3 ('HS', '104B') :  enrolled:  175
#4 ('PSYC', '10A') :  enrolled:  166
#5 ('COSI', '164A') :  enrolled:  166
#6 ('LGLS', '10A') :  enrolled:  145
#7 ('ENVS', '2A') :  enrolled:  141
#8 ('BIOL', '16A') :  enrolled:  136
#9 ('NPSY', '199A') :  enrolled:  135
#10 ('PSYC', '34B') :  enrolled:  119
#11 ('BIOL', '350A') :  enrolled:  105
#12 ('COSI', '21A') :  enrolled:  102
#13 ('ANTH', '1A') :  enrolled:  101
#14 ('COSI', '29A') :  enrolled:  101
#15 ('PSYC', '31A') :  enrolled:  96
#16 ('PSYC', '33A') :  enrolled:  96
#17 ('COSI', '131A') :  enrolled:  94
#18 ('BIOL', '350B') :  enrolled:  92
#19 ('SOC', '83A') :  enrolled:  92
#20 ('SOC', '84A') :  enrolled:  90


### i) Question 1: how many courses offered last year had students on the waitlist?

In [35]:
wait_listed = 0
for c in courses:
    if c['waiting'] > 0:
        wait_listed += 1
wait_listed

214

### i) Question 2: which BIO class had the most students enrolled?

In [36]:
bio_enrolled = {}
for c in courses:
    if c['subject'] == "BIOL":
        bio_enrolled[c['coursenum']] = c['enrolled']

In [37]:
max_course = max(bio_enrolled, key=bio_enrolled.get)
max_course

'42A'

### i) Question 3: which COSI courses offered last year had zero students enrolled?

In [38]:
zero_enrolled = set()
for c in courses:
    if c['subject'] == 'COSI' and c['enrolled'] == 0:
        zero_enrolled.add(c['name'])
zero_enrolled

{'Computational Linguistics Capstone Project',
 "Computational Linguistics Master's Thesis",
 'Computational Linguistics Research Internship',
 'Dissertation Research',
 'Graduate Research Internship',
 'Independent Study',
 "Master's Project",
 "Master's Research Internship",
 'Readings',
 'Research Internship and Analysis',
 'Senior Field Project',
 'Senior Research'}

### i) Question 4: Question 4: are there any faculty who tought both a COSI and LING course last year?

In [39]:
cosi_profs = set()
ling_profs = set()
for c in courses:
    if c['subject'] == "COSI":
        cosi_profs.add(c['instructor'][2])
    if c['subject'] == "LING":
        ling_profs.add(c['instructor'][2])

In [40]:
if bool(set(cosi_profs) & set(ling_profs)):
    print("These profs teach both cosi and ling courses:", "\n", cosi_profs & ling_profs)
else:
    print("No common profs!")

These profs teach both cosi and ling courses: 
 {'smalamud@brandeis.edu', 'jamesp@brandeis.edu', 'lmgold@brandeis.edu'}


### Ben is doing mardown in jupyter right now

### Hannah is doing markdown in jupyter right now